In [41]:
import matplotlib
import re
import os
import sys

##Current Step: Start plotting connection indexes by IP Header Size
##Current Step: Start writing out packet_strings to files separated by connection index.

In [42]:
capture_src = 'captures'
capture_dst = 'captures_split'
processed = 'processed_log.txt'

In [47]:
class connection_map:
    connection_list = list()
    def __init__(self):
        self.connection_list = list()
    def get_connection_index(self,packet):
        connection_string = "%s:%s-%s:%s-%s"%(packet.src_host,packet.src_port,packet.dst_host,packet.dst_port,packet.proto)
        try:
            connection_index = self.connection_list.index(connection_string)
        except ValueError:
            self.connection_list.append(connection_string)
            return self.connection_list.index(connection_string)


In [48]:
class packet:
    packet_string = ""
    src_host = ""
    src_port = ""
    dst_host = ""
    dst_port = ""
    proto = ""
    def __init__(self,src_host,src_port,dst_host,dst_port,proto,packet_string):
        self.src_host = src_host
        self.src_port = src_port
        self.dst_host = dst_host
        self.dst_port = dst_port
        self.proto = proto
        self.packet_string = packet_string
    def to_string(self):
        print("srcHost:%s\nsrcPort:%s\ndstHost:%s\ndstPort:%s\nproto:%s\n" % (self.src_host,self.src_port,self.dst_host,self.dst_port,self.proto))

In [51]:
def get_packets_in_file(capture_file):
    '''
    packet_capture_file = open(capture_file,'r')
    packet_capture_records = packet_capture_file.read()
    packet_capture_file.close()
    packets = re.findall(".*IP.*length.*Flags.*length.*\n",packet_capture_records,re.MULTILINE)
    '''
    #debug
    temp = 0
    
    line_one = ''
    line_two = ''
    packet_connection_map = connection_map()
    with open(capture_file,'r') as src:
        for line in src:
            line_one = line_two
            line_two = line
            line_pair = line_one + line_two
            
            packet_match = re.match(r'.*IP.*proto\W(?P<proto>\w+).*length.*\r?\n\W*(?P<srcHost>[\w\.]+)\.(?P<srcPort>[\w]+)\W\>\W(?P<dstHost>[\w\.]+)\.(?P<dstPort>[\w]+).*Flags.*length.*',line_pair,re.MULTILINE)
            if(packet_match):
                cur_packet = packet(packet_match.group('srcHost'),packet_match.group('srcPort'),packet_match.group('dstHost'),packet_match.group('dstPort'),packet_match.group('proto'),line_pair)
                connection_idx = packet_connection_map.get_connection_index(cur_packet)
                
                print("Connection_Index:%s"%connection_idx)
                print(line_pair)
                print(cur_packet.to_string())
                raise KeyboardInterrupt
            #Debug Begin
            temp = temp +1
            if temp == 2:
                print("Your regex didn't match nub!!!!")
                print(line_pair)
                raise KeyboardInterrupt
            #Debug End
    
    return packets

In [52]:
try:
    for x in os.walk(capture_src): #each os.walk element is: dirpath, subdir-names, dir-filenames
        outdir = x[0].replace(capture_src,capture_dst)
        try:
            os.mkdir(outdir)
        except FileExistsError:
            pass #We don't care if the directory already exists
        for capture in x[2]:
            if capture.endswith(".cap"):
                base_file = capture[:-4]
                date_file = base_file + '.date'
                capture_packets = get_packets_in_file(x[0]+"\\"+capture)
                
                for packet in capture_packets:
                    print(packet)
                    raise KeyboardInterrupt

                        
except KeyboardInterrupt:
    pass
            
    #print(outdir)

Connection_Index:0
2018-09-30 00:27:33.133058 IP (tos 0x10, ttl 64, id 40973, offset 0, flags [DF], proto TCP (6), length 104)
    ShadowSense.shadowdomain.ssh > Shadowscout.shadowdomain.52303: Flags [P.], cksum 0xb439 (correct), seq 2878089682:2878089746, ack 1345784929, win 513, length 64

srcHost:ShadowSense.shadowdomain
srcPort:ssh
dstHost:Shadowscout.shadowdomain
dstPort:52303
proto:TCP

None


In [53]:
#Isolate the outputs into <src_name>.connectionNumber.cap
##Then plot the connections onto a single graph.
###The dominant connection in a packet capture is the one you keep